In [ ]:
# GENERATE docs/chars.txt TABLE OF ORDINALS => SYMBOL MAPPINGS
from benchmark.util import UniUtil

lines = [""]
for i in range(32, 2**16):
    content = (UniUtil.chr(i).encode("utf-8").rjust(4) + " => " + str(i).ljust(7)).ljust(4+4+7)
    if len(lines[-1]) + len(content) <= 120:
        lines[-1] += content
    else:
        lines[-1] += "\n"
        lines.append(content)



file = open("docs/chars.txt", "w")
file.write(reduce(lambda p,c: p+c, lines, ""))
file.close()

In [1]:
# FAdo BACKTRACKING DEMO
from benchmark.convert import Converter

evil = "(a + a)*"
re = Converter().math(evil)

print "iter derivative backtrack"
for i in range(30):
    word = "a"*i + "b"
    print str(i).ljust(4),
    print str(re.evalWordP(word)).ljust(10),
    print re.evalWordPBacktrack(word)

iter derivative backtrack
0    False      False
1    False      False
2    False      False
3    False      False
4    False      False
5    False      False
6    False      False
7    False      False
8    False      False
9    False      False
10   False      False
11   False      False
12   False      False
13   False      False
14   False      False
15   False      False
16   False      False
17   False      False
18   False      False
19   False      False
20   False      False
21   False      False
22   False     

In [ ]:
# CATASTROPHIC BACKTRACKING ISSUE IN PYTHON'S RE MODULE
import re
expr = re.compile("(a|aa)*b+")
print expr.search("a"*35 + "c")